In [3]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time
pd.set_option("max_columns", 200)





year = "2020"
startDate = '20200103'
endDate = '20200103'
readPath = 'L:\\DB\\' + year + '\\SH\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

for data in dataPathLs:
    
    readPath = data + '\\snapshot\\Level_2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[((dateLs >= 600000) & (dateLs <= 700000))]
    SH = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i, usecols = [0,1,3,5,7,9,10,11,15,17,18,19,20,21,22,23,25,26,28,29,30,31,32,33,37,39,40,41,
                                          42,43,46,47,49,50])
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        SH += [df]
    del df
    SH = pd.concat(SH).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    SH["skey"] = SH["StockID"] + 1000000
    SH["date"] = int(SH["QuotTime"].iloc[0]//1000000000)
    SH["time"] = (SH['QuotTime'] - int(SH['QuotTime'].iloc[0]//1000000000*1000000000)).astype(np.int64) * 1000
    SH["clockAtArrival"] = SH["QuotTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    SH['datetime'] = SH["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    print(datetime.datetime.now() - startTm)
    
    
    startTm = datetime.datetime.now()
    SH["BidPrice"] = SH["BidPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
    SH["OfferPrice"] = SH["OfferPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
    SH["BidOrderQty"] = SH["BidOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SH["OfferOrderQty"] = SH["OfferOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SH["BidNumOrders"] = SH["BidNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SH["OfferNumOrders"] = SH["OfferNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

    for i in range(1, 11):
        SH["bid" + str(i) + 'p'] = SH["BidPrice"].apply(lambda x: x[i-1])
        SH["ask" + str(i) + 'p'] = SH["OfferPrice"].apply(lambda x: x[i-1])
        SH["bid" + str(i) + 'q'] = SH["BidOrderQty"].apply(lambda x: x[i-1])
        SH["ask" + str(i) + 'q'] = SH["OfferOrderQty"].apply(lambda x: x[i-1])
        SH["bid" + str(i) + 'n'] = SH["BidNumOrders"].apply(lambda x: x[i-1])
        SH["ask" + str(i) + 'n'] = SH["OfferNumOrders"].apply(lambda x: x[i-1])
    
    SH["BidOrders"] = SH["BidOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SH["OfferOrders"] = SH["OfferOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

    for i in range(1, 51):
        SH["bid1Top" + str(i) + 'q'] = SH["BidOrders"].apply(lambda x: x[i-1])
        SH["ask1Top" + str(i) + 'q'] = SH["OfferOrders"].apply(lambda x: x[i-1])
    print(datetime.datetime.now() - startTm)
    
    
    startTm = datetime.datetime.now()
    SH = SH.rename(columns={"NumTrades":"cum_trades_cnt", "HighPx":"high", "OpenPx":"open", "PreClosePx":"prev_close", "LowPx":"low",
                            "WeightedAvgBidPx":"total_bid_vwap", "WithdrawSellNumber":"cum_canceled_sell_orders", "TotalOfferNumber":"total_ask_orders",
                           "OfferTradeMaxDuration":"ask_trade_max_duration", "TotalBidNumber":"total_bid_orders", "WithdrawSellMoney":"cum_canceled_sell_amount",
                           "TotalOfferQty":"total_ask_quantity", "WithdrawBuyNumber":"cum_canceled_buy_orders", "WeightedAvgOfferPx":"total_ask_vwap",
                           "WithdrawSellAmount":"cum_canceled_sell_volume", "Volume":"cum_volume", "NumOfferOrders":"total_ask_levels", "TotalBidQty":"total_bid_quantity",
                           "WithdrawBuyAmount":"cum_canceled_buy_volume", "LastPx":"close", "BidTradeMaxDuration":"bid_trade_max_duration", 
                           "NumBidOrders":"total_bid_levels", "Amount":"cum_amount", "WithdrawBuyMoney":"cum_canceled_buy_amount"})
    
    SH = SH.fillna(0)
    SH = SH[~((SH["bid1p"] == 0) & (SH["ask1p"] == 0))]
    SH["ordering"] = SH.groupby("skey").cumcount()
    SH["ordering"] = SH["ordering"] + 1
    
    for col in ["cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q', "total_bid_quantity", "total_ask_quantity",'cum_canceled_buy_volume',
        'cum_canceled_sell_volume']:
        SH[col] = SH[col].astype('int64')
    
    SH["has_missing"] = 0
    
    for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
        'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders','cum_canceled_sell_orders',
            "ordering", 'bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n', 'bid9n', 'bid10n', 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n',
        'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bid_trade_max_duration', 'ask_trade_max_duration',
               'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', 'has_missing']:
        SH[col] = SH[col].astype('int32')
    
    for cols in ["prev_close", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p']:
        SH[cols] = SH[cols].apply(lambda x: round(x, 2)).astype(np.float32)
    
    for cols in ['cum_amount', "cum_canceled_sell_amount", "cum_canceled_buy_amount"]:
        SH[cols] = SH[cols].apply(lambda x: round(x, 2)).astype(np.float64)
        
    for cols in ['total_bid_vwap', "total_ask_vwap"]:
        SH[cols] = SH[cols].apply(lambda x: round(x, 3)).astype(np.float64)
        
    display(SH["date"].iloc[0])
    display(SH.groupby("skey")["time"].min().max())
        
        
    assert(sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    SH["prev_close"] = np.where(SH["time"] >= 91500000000, SH.groupby("skey")["prev_close"].transform("max"), SH["prev_close"]) 
    SH["open"] = np.where(SH["cum_volume"] > 0, SH.groupby("skey")["open"].transform("max"), SH["open"])
    assert(sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    assert(SH[SH["cum_volume"] > 0]["open"].min() > 0)
        
    SH = SH[["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume", "cum_amount", "prev_close",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"total_bid_quantity", "total_ask_quantity","total_bid_vwap", "total_ask_vwap",
        "total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration', 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
        "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount"]]
    print(datetime.datetime.now() - startTm)
    print(SH["date"].iloc[0] + " SZ finished")
    del SH
    
    
    
    
    
readPath = 'L:\\DB\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

for data in dataPathLs:    
    
    readPath = data + '\\snapshot\\Level_2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    SZ = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i, usecols = [0,1,4,5,6,7,9,12,17,18,19,24,25,26,28,29,30,32,33,34,35])
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        SZ += [df]
    del df
    SZ = pd.concat(SZ).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    SZ["skey"] = SZ["StockID"] + 2000000
    SZ["date"] = int(SZ["QuotTime"].iloc[0]//1000000000)
    SZ["time"] = (SZ['QuotTime'] - int(SZ['QuotTime'].iloc[0]//1000000000*1000000000)).astype(np.int64) * 1000
    SZ["clockAtArrival"] = SZ["QuotTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    SZ['datetime'] = SZ["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    SZ["BidPrice"] = SZ["BidPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
    SZ["OfferPrice"] = SZ["OfferPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
    SZ["BidOrderQty"] = SZ["BidOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SZ["OfferOrderQty"] = SZ["OfferOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SZ["BidNumOrders"] = SZ["BidNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SZ["OfferNumOrders"] = SZ["OfferNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

    for i in range(1, 11):
        SZ["bid" + str(i) + 'p'] = SZ["BidPrice"].apply(lambda x: x[i-1],2)
    del SZ["BidPrice"]
    print("1")
    for i in range(1, 11):
        SZ["ask" + str(i) + 'p'] = SZ["OfferPrice"].apply(lambda x: x[i-1],2)
    del SZ["OfferPrice"]
    print("2")
    for i in range(1, 11):
        SZ["bid" + str(i) + 'q'] = SZ["BidOrderQty"].apply(lambda x: x[i-1])
    del SZ["BidOrderQty"]
    print("3")
    for i in range(1, 11):
        SZ["ask" + str(i) + 'q'] = SZ["OfferOrderQty"].apply(lambda x: x[i-1])
    del SZ["OfferOrderQty"]
    print("4")
    for i in range(1, 11):
        SZ["bid" + str(i) + 'n'] = SZ["BidNumOrders"].apply(lambda x: np.int32(x[i-1]))
    del SZ["BidNumOrders"]
    print("5")
    for i in range(1, 11):
        SZ["ask" + str(i) + 'n'] = SZ["OfferNumOrders"].apply(lambda x: np.int32(x[i-1]))
    del SZ["OfferNumOrders"]
    print("6")
    
    SZ["BidOrders"] = SZ["BidOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
    SZ["OfferOrders"] = SZ["OfferOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

    for i in range(1, 51):
        SZ["bid1Top" + str(i) + 'q'] = SZ["BidOrders"].apply(lambda x: np.int32(x[i-1]))
    del SZ["BidOrders"]
    print("7")
    
    for i in range(1, 51):
        SZ["ask1Top" + str(i) + 'q'] = SZ["OfferOrders"].apply(lambda x: np.int32(x[i-1]))
    del SZ["OfferOrders"]
    print("8")
    print(datetime.datetime.now() - startTm)
        
    
    startTm = datetime.datetime.now()
    SZ = SZ.rename(columns={"NumTrades":"cum_trades_cnt", "TotalBidQty":"total_bid_quantity", "TotalOfferQty":"total_ask_quantity", 
                           "HighPx":"high", "OpenPx":"open", "PreClosePx":"prev_close", "LowPx":"low", "LastPx":"close", "WeightedAvgBidPx":"total_bid_vwap", 
                           "WeightedAvgOfferPx":"total_ask_vwap", "Amount":"cum_amount", "Volume":"cum_volume"})
    
    SZ = SZ.fillna(0)
    SZ = SZ[~((SZ["bid1p"] == 0) & (SZ["ask1p"] == 0))]
    SZ["ordering"] = SZ.groupby("skey").cumcount()
    SZ["ordering"] = SZ["ordering"] + 1

    for cols in ["total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration',
                 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume', "cum_canceled_buy_amount",
                 "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount", "has_missing"]:
        SZ[cols] = 0
        
    for col in ["cum_volume", "total_bid_quantity", "total_ask_quantity",'cum_canceled_buy_volume',
        'cum_canceled_sell_volume']:
        SZ[col] = SZ[col].astype('int64')
    
    for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
        'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders','cum_canceled_sell_orders',
            "ordering", 'bid_trade_max_duration', 'ask_trade_max_duration', "has_missing"]:
        SZ[col] = SZ[col].astype('int32')
    
        
    for cols in ["prev_close", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p']:
        SZ[cols] = SZ[cols].apply(lambda x: round(x, 2)).astype(np.float32)

        
    for cols in ['cum_amount', "cum_canceled_sell_amount", "cum_canceled_buy_amount"]:
        SZ[cols] = SZ[cols].apply(lambda x: round(x, 2)).astype(np.float64)
    
            
    display(SZ["date"].iloc[0])
    display(SZ.groupby("skey")["time"].min().max())
        
        
    assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    SZ["prev_close"] = np.where(SZ["time"] >= 91500000000, SZ.groupby("skey")["prev_close"].transform("max"), SZ["prev_close"]) 
    SZ["open"] = np.where(SZ["cum_volume"] > 0, SZ.groupby("skey")["open"].transform("max"), SZ["open"])
    assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    assert(SZ[SZ["cum_volume"] > 0]["open"].min() > 0)
        
    SZ = SZ[["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume", "cum_amount", "prev_close",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"total_bid_quantity", "total_ask_quantity","total_bid_vwap", "total_ask_vwap",
        "total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration', 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
        "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount"]]
    print(datetime.datetime.now() - startTm)
    print(SZ["date"].iloc[0] + " SZ finished")
    del SZ
    
    
    

20200103

92503000000

0:04:10.613453


KeyboardInterrupt: 

In [9]:
SZ.dtypes

skey                                 int32
date                                 int32
time                                 int64
clockAtArrival                       int64
datetime                    datetime64[ns]
ordering                             int32
cum_trades_cnt                       int32
cum_volume                           int64
cum_amount                         float64
prev_close                         float32
open                               float32
high                               float32
low                                float32
close                              float32
bid10p                             float32
bid9p                              float32
bid8p                              float32
bid7p                              float32
bid6p                              float32
bid5p                              float32
bid4p                              float32
bid3p                              float32
bid2p                              float32
bid1p      

In [6]:
pd.set_option("max_rows", 200)
SZ.dtypes

skey                                 int32
date                                 int32
time                                 int64
clockAtArrival                       int64
datetime                    datetime64[ns]
ordering                             int32
cum_trades_cnt                       int32
cum_volume                           int64
cum_amount                         float64
prev_close                         float32
open                               float32
high                               float32
low                                float32
close                              float32
bid10p                             float32
bid9p                              float32
bid8p                              float32
bid7p                              float32
bid6p                              float32
bid5p                              float32
bid4p                              float32
bid3p                              float32
bid2p                              float32
bid1p      

In [76]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 80
__builtins__ 80
_ih 768
_oh 2280
_dh 72
In 768
Out 2280
get_ipython 64
exit 56
quit 56
_ 1487937768
__ 192
___ 1487937768
_i 63
_ii 66
_iii 63
_i1 21931
pd 80
random 80
np 80
glob 80
os 80
rarfile 80
py7zr 80
pickle 80
datetime 80
time 80
year 53
startDate 75
endDate 75
readPath 92
dataPathLs 454160
dateLs 13036
data 131
SZ 29691118944
ll 64
startTm 48
i 53
_i2 56
_2 64
_i3 58
_i4 73
sys 80
_i5 77
_5 32
_i6 91
_6 28
_i7 54
_7 488
_i8 65
_8 48
_i9 61
_i10 68
_10 28
_i11 95
_i12 68
_12 28
_i13 66
_13 32
_i14 54
_14 53
_i15 733
psutil 80
info 88
_i16 54
_16 624
_i17 53
_17 149
_i18 52
_18 21931
_i19 51
_19 768
_i20 62
_i21 66
_21 28
_i22 67
_22 28
_i23 65
_23 28
_i24 66
_24 28
_i25 75
_25 28
_i26 67
_26 28
_i27 68
_27 28
_i28 74
_28 28
_i29 68
_29 28
_i30 70
_30 28
_i31 72
_31 28
_i32 68
_32 28
_i33 72
_33 28
_i34 66
_34 32
_i35 66
_35 28
_i36 67
_36 28
_i37 65
_37 28
_i38 67
_38 28
_i39 67
_39 28
_i40 66
_40 

In [16]:
SZ["BidPrice"].iloc[1000]

'[12.21,12.2,12.19,12.18,12.17,12.16,12.15,12.14,12.13,12.12]'

In [17]:
SZ["bid1p"].iloc[1000]

'12.21'

In [ ]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

In [72]:
import psutil
import os

info = psutil.virtual_memory()
print(u'内存使用：',psutil.Process(os.getpid()).memory_info().rss)
print(u'总内存：',info.total)
print(u'内存占比：',info.percent)
print(u'cpu个数：',psutil.cpu_count())

内存使用： 41984040960
总内存： 68625629184
内存占比： 75.4
cpu个数： 8


In [6]:
SZ["vwapAsk"].astype(str).apply(lambda x: len(str(x.split('.')[1]))).unique()

array([1, 2], dtype=int64)

In [11]:
pd.set_option("max_rows", 300)
SZ.dtypes

skey                                 int32
date                                 int32
time                                 int64
clockAtArrival                       int64
datetime                    datetime64[ns]
ordering                             int32
cum_trades_Cnt                       int32
cum_volume                           int64
cum_amount                         float64
prevClose                          float32
open                               float32
high                               float32
low                                float32
close                              float32
bid10p                             float32
bid9p                              float32
bid8p                              float32
bid7p                              float32
bid6p                              float32
bid5p                              float32
bid4p                              float32
bid3p                              float32
bid2p                              float32
bid1p      

In [12]:
test = SZ[SZ["time"] > 93000000000].head(1)
test

,skey,date,time,clockAtArrival,datetime,ordering,cum_trades_Cnt,cum_volume,cum_amount,prevClose,open,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid10n,bid9n,bid8n,bid7n,bid6n,bid5n,bid4n,bid3n,bid2n,bid1n,ask1n,ask2n,ask3n,ask4n,ask5n,ask6n,ask7n,ask8n,ask9n,ask10n,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,total_Bid_Quantity,total_Ask_Quantity,vwapBid,vwapAsk,total_Bid_Orders,total_Ask_Orders,total_Bid_Levels,total_Ask_Levels,bid_Trade_Max_Duration,ask_Trade_Max_Duration,cum_canceled_Buy_Orders,cum_canceled_Buy_Volume,cum_canceled_Buy_Amount,cum_canceled_Sell_Orders,cum_canceled_Sell_Volume,cum_canceled_Sell_Amount
101,2000001,20170906,93003000000,1504661403000000,2017-09-06 09:30:03,56,354,1324188,15334010.4,11.64,11.59,11.6,11.55,11.56,11.47,11.48,11.49,11.5,11.51,11.52,11.53,11.54,11.55,11.56,11.57,11.58,11.59,11.6,11.61,11.62,11.63,11.64,11.65,11.66,34300,67100,36100,194400,397000,59400,72300,12400,140212,167600,300,60800,76940,206500,5000,28060,100300,33000,600,860,12,23,18,93,66,28,19,7,25,19,1,8,24,5,1,11,3,14,2,2,9100,700,120000,1000,500,500,500,1000,21000,5000,200,2000,100,300,100,1300,1300,1000,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4880112,7865226,11.37,12.09,0,0,0,0,0,0,0,0,0.0,0,0,0.0


In [76]:
datetime.datetime.fromtimestamp(1504661401000000/1e6).strftime("%Y-%m-%d %H:%M:%S %f")

'2017-09-06 09:30:01 000000'